In [130]:
%load_ext tensorboard
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
import datetime
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
2.3.1


In [51]:
prefix = "/mnt/data/s3/cp2020/"
#dataset = pd.read_excel("Data set много букв — копия (2) (1).xlsx")
#dataset.to_csv('train.csv',encoding="utf8")
dataset = pd.read_csv('train.csv',encoding="utf8")


In [52]:
dataset = dataset.fillna(1,axis=1)
cols = list(dataset.columns)
dataset[cols] = dataset[cols].apply(pd.to_numeric,errors='coerce')
#dataset = dataset.apply()

#dataset = pd.to_numeric(dataset)
dataset = dataset.astype(int)
dataset['Энергопотребление (анализируемый показатель), млн. кВт в час']
len(list(dataset.columns))
dataset.isnull().values.any()

False

In [53]:
train_dataset = dataset.sample(frac=0.7,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
#list(train_dataset.columns)
#train_dataset.to_csv("test.csv")

In [54]:
train_labels = train_dataset.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')
test_labels = test_dataset.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')

In [55]:
#list(train_dataset.columns)

train_stats = train_dataset.describe()
#train_stats.pop('Энергопотребление (анализируемый показатель), млн. кВт в час')
train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,2557.0,1815.473211,1062.912804,1.0,904.0,1802.0,2734.0,3652.0
"ВРП, рублей",2557.0,68570.544779,29095.786823,0.0,57815.0,68280.0,89231.0,108882.0
Белгородская область,2557.0,1004.126711,411.706409,0.0,906.0,1010.0,1287.0,1532.0
Брянская область,2557.0,477.988659,207.714778,0.0,375.0,481.0,629.0,747.0
Владимирская область,2557.0,584.037153,241.398632,0.0,498.0,592.0,774.0,879.0
...,...,...,...,...,...,...,...,...
Амурская область.9,2557.0,19.000000,0.000000,19.0,19.0,19.0,19.0,19.0
Магаданская область.8,2557.0,5.000000,0.000000,5.0,5.0,5.0,5.0,5.0
Сахалинская область.8,2557.0,10.000000,0.000000,10.0,10.0,10.0,10.0,10.0
Еврейская автономная область.7,2557.0,3.000000,0.000000,3.0,3.0,3.0,3.0,3.0


In [56]:
#def norm(x):
#  return (x - train_stats['mean']) / train_stats['std']
#normed_train_data = norm(train_dataset)
#normed_test_data = norm(test_dataset)
#normed_train_data

In [133]:
def build_model():
  model = keras.Sequential([
    keras.layers.Flatten(input_shape=[len(train_dataset.keys())]),
    layers.Dense(50,activation='relu'),
    layers.Dense(100,activation='softmax'),
    layers.Dense(150,activation='softmax'),
    layers.Dense(1)
  ])
  
    
  optimizer = tf.keras.optimizers.RMSprop(0.1)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'],)
  return model


In [134]:
model = build_model()

In [135]:
model.summary()


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_14 (Flatten)         (None, 830)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 50)                41550     
_________________________________________________________________
dense_47 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_48 (Dense)             (None, 150)               15150     
_________________________________________________________________
dense_49 (Dense)             (None, 1)                 151       
Total params: 61,951
Trainable params: 61,951
Non-trainable params: 0
_________________________________________________________________


In [143]:
example_batch = train_dataset[:10]
#example_batch
example_result = model.predict(example_batch)
example_result

array([[1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217],
       [1283.2217]], dtype=float32)

In [145]:
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 100

history = model.fit(
  train_dataset, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[PrintDot()])


..........................................

KeyboardInterrupt: 

In [146]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
log_dir =  datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_dataset, train_labels, epochs=EPOCHS,
                    validation_split = 0.2, verbose=0, callbacks=[tensorboard_callback])


In [148]:
%tensorboard --logdir .

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [120]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()


,loss,mae,mse,val_loss,val_mae,val_mse,epoch
95,2466085.00,1570.372681,2466085.00,2445686.75,1563.869141,2445686.75,95
96,2426081.25,1557.583496,2426081.25,2405849.50,1551.080078,2405849.50,96
97,2386413.50,1544.797852,2386413.50,2366360.50,1538.298218,2366360.50,97
98,2347088.00,1532.016479,2347088.00,2327201.75,1525.516968,2327201.75,98
99,2308089.75,1519.235107,2308089.75,2288368.50,1512.735718,2288368.50,99


In [121]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)
print(mae)

35/35 - 0s - loss: nan - mae: nan - mse: nan
nan


In [122]:
test_predictions = model.predict(test_dataset).flatten()

test_predictions

array([1283.2639, 1283.2644, 1283.2644, ..., 1283.2644, 1283.2644,
       1283.2644], dtype=float32)